In [5]:
import os
from scripts.Unitex import Unitex
from scripts.posTagger_to_unitex import spacy2unitex
import spacy

In [ ]:
# pip install -U spacy
#!python -m spacy download en_core_web_sm
#!python -m spacy download fr_core_news_sm

## Create Unitex input txt file from spaCy doc

In [6]:
# Load French tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")

# Process whole documents
text = ("I am visiting the city of Zurich.")
doc = nlp(text)

unitex_input = spacy2unitex(doc)
print(unitex_input)
filename = 'tmp'
filepath = os.path.join('output', filename + '.txt')    
with open(filepath, 'w') as f:
    f.write(unitex_input)


{I,I.PRON} {am,be.AUX} {visiting,visit.VERB} {the,the.DET} {city,city.NOUN} {of,of.ADP} {Zurich,Zurich.PROPN} {.,.PUNCT} 


## Run Unitex as a Python code snippet

In [ ]:
version = "Landscape_sentiment"
lang = 'English'
install_path = "/Users/lmoncla/Nextcloud-LIRIS/Programmes/Unitex-GramLab-3.2"
install_path_app = "/Users/lmoncla/Programmes/Unitex-GramLab-3.2/App"
filepath = os.path.join('output', filename)

delete_tmp_files = True

unitex = Unitex(version, lang, install_path, install_path_app, delete_tmp_files)
doc = unitex.run(filepath)


In [ ]:
doc

## Run `Unitex.py` as a Python script

In [ ]:
version = "Landscape_sentiment"
lang = 'English'
install_path = "/Users/lmoncla/Nextcloud-LIRIS/Programmes/Unitex-GramLab-3.2"
install_path_app = "/Users/lmoncla/Programmes/Unitex-GramLab-3.2/App"
filepath = os.path.join('output', filename)

!python scripts/Unitex.py -i $filepath -l $lang -c $version --install_path $install_path --install_path_app $install_path_app

with open(filepath + "_csc_csc.xml", "r") as file:
    doc = file.read()


## Parse the XML result